一个普通欧式期权，在期末期权买方的现金流是：max（S-K，0）。这导致了一个问题：期权的未来现金流是不确定的（各位可以参照看看债券的定价，其未来的现金流就是确认的）。不确定的现金流，为定价带来了困难

在期权定价中，为了解决第一个定价核心（现金流确认），我们需要引入：**概率**。即未来某个时刻标的价格S大于执行价K的概率

站在某个时点看，S在下一个时点可能出现的结果是无穷多的。于是，我们可以通过引入一个<font color = red>关于S的微小（时间）变动</font>，规定其波动的
度但不规定其走向，从而解决标的S的随机波动问题

为了确认现金流，我们引入了概率。但是引入概率会为第二个定价核心带来新的问题：由于随机源（即S的变动方向）的存在，使得在期权范畴中的一价定律无法施展。

随机源导致一价定律无法施展，那么反过来想，只要随机源被消除了，那么一价定律就是可以用了,通过对冲，把S的随机项消除掉，从而使得一价定律可用

期权中的一价定律的运用，就是在对冲过后，剩下的收益，只能是无风险的。
$$\frac{1}{2} \operatorname{\Gamma}\left(dS\right)^2+$ \Theta dt +
\sigma_{IV}Vega$=r \pi dt$$
> BSM框架下不存在Vega项，因为假设了IV恒定

而Theta又近似等于：
$$Theta \approx \frac{1}{2} \Gamma \sigma_{IV}^2 S_t^2 $$
> 这里的$\sigma_{IV}$是购入/卖出期权时，对应的IV

所以：
$$\Delta PnL = \frac{1}{2} S^2 \Gamma ((\frac{\Delta S}{S})^2 - \sigma_{IV}^2 \Delta t) + \Delta \sigma_{IV} * Vega$$
$$ = \frac{1}{2} S^2 \Gamma( (\frac{\Delta S}{S})^2 / \Delta t - \sigma_{IV}^2 ) + \Delta \sigma_{IV} * Vega$$
$$ = \frac{1}{2} S^2 \Gamma (RV^2 - \sigma_{IV}^2 ) + \Delta \sigma_{IV} * Vega$$
> 需注意的是，这里的波动率和时间都是**年化值**

如果再加上Vega的话，Vega和Gamma之间有：
$$ Vega = \Gamma \sigma_{IV} S^2 * TTM$$
这个公式意味着Gamma和Vega的正负号一致。在**Vega相等**的前提下，**到期日越近**Gamma越大、IV越低Gamma越大；在**相等Gamma**的前提下，到期日越远或IV越高的时候，Vega就越大

将此式代入后，期权的pnl变为：
$$\Delta PnL = Vega (RV - IV)\Delta t/T$$
> 这里的T不是到期时间，而是一段时间间隔

所以从期初到现在的**平均**$Cumulative PnL = \sum{\Delta PnL} = Vega * (RV-IV)$


In [ ]:
import numpy as np
import pandas as pd
from monte_carlo.monte_carlo_class1 import MonteCarloOptionPricing
from py_vollib_vectorized import vectorized_black_scholes,get_all_greeks
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
S0 = 50  # e.g. spot price = 35
K = 52.08  # e.g. exercise price = 40
T = 0.5  # e.g. one year
r = 0.05  # e.g. risk free rate = 1%
sigma = 0.3  # e.g. volatility = 5%
div_yield = 0.0  # e.g. dividend yield = 1%
no_of_slice = 91  # no. of slices PER YEAR e.g. quarterly adjusted

In [ ]:
MC = MonteCarloOptionPricing(S0=S0,
                             K=K,
                             T=T,
                             r=r,
                             sigma=sigma,
                             div_yield=div_yield,
                             simulation_rounds=int(10000),
                             no_of_slices=no_of_slice,
                             # fix_random_seed=True,
                             fix_random_seed=500)

In [ ]:
MC.stock_price_simulation()
price_array = MC.price_array

plt.figure(figsize=(10, 6))
for i in (range(len(price_array))):
    plt.plot(price_array[i,:])

plt.title('MC')
plt.xlabel('period')
plt.ylabel('St')

plt.show()

In [ ]:
nohedge_pnl = np.zeros([10000,1])
hedge_pnl = np.zeros([10000,1])
for i in range(len(price_array)):
    price = price_array[i,:]
    option_price = vectorized_black_scholes('c',price,K,T,r,sigma)
    option_greeks = get_all_greeks('c',price,K,T,r,sigma)
    df = pd.DataFrame((price),columns=['s'])
    df = pd.concat([df,option_price,option_greeks],axis=1)

    nohedge = (option_price - option_price.shift()).sum()['Price']
    nohedge_pnl[i] = nohedge

    hedge = []
    for j in range(1,len(df)):
        hedge.append(0.5 * (df.loc[j,'gamma'] + df.loc[j-1,'gamma']) * 0.5 * (df.loc[j,'s'] - df.loc[j-1,'s'])**2 + df.loc[j-1,'theta'])
    hedge_pnl[i] = sum(hedge)

In [ ]:
nohedge_pnl

In [ ]:
hedge_pnl

In [ ]:
fig = plt.figure(figsize=(8,5))
fig.add_subplot(1,2,1)
plt.hist(nohedge_pnl,label='no hedge pnl')
fig.add_subplot(1,2,2)
plt.hist(hedge_pnl,label='hedged pnl')

可发现在对冲过后，pnl的分布开始向正态分布靠拢，相当于把未对冲情况下的pnl的分布**平摊**到了整个x轴上